In [89]:
# %%capture

import sys
import os
import importlib
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'import'))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'module'))

import import_default
import import_database
import import_other

importlib.reload(import_default)
importlib.reload(import_database)
importlib.reload(import_other)

from import_default import *
from import_database import *
from import_other import *

In [90]:
stock_dict = {
    'Vượt trội': ["ANV", "CEO", "CII", "DXG", "DXS", "EVG", "GEX", "MBB", "MSB", "ORS", "PDR", "SHB", "SHS", "SSI", "TPB", "VOS", "VHM", "VIC", "VIX", "VND", "VPB", "VSC"],
    'Tốt': ["ACB", "AGR", "APS", "BSR", "CTG", "CTS", "DIG", "DPM", "DSE", "EIB", "EVF", "FCN", "HAG", "HDB", "HDC", "HDG", "HHS", "HHV", "KBC", "KDH", "LCG", "LPB", "MBS", "NVL", "OCB", "SCR", "STB", "TCB", "TCH", "VGC", "VIB", "VIC", "VRE"],
    'Lưu ý': ["BID", "BSI", "DCM", "DPG", "GMD", "HAH", "HCM", "HPG", "HUT", "IJC", "MSN", "MWG", "NKG", "NLG", "PAN", "PET", "POW", "PVD", "VCG", "VCI", "VFS", "VEG"]
}

In [91]:
full_stock_classification_df = get_mongo_collection(ref_db, 'full_stock_classification')
current_quarter_classification_df = get_mongo_collection(ref_db, 'current_quarter_classification')
date_series = get_mongo_collection(ref_db, 'date_series')
today = date_series.iloc[0]['date']

projection={"_id": 0, 'date': 1, 'ticker': 1, 'vol_ratio': 1, 'W_MTSI': 1, 'M_MTSI': 1, 'Q_MTSI': 1, 'Y_MTSI': 1, 'W_MRVI': 1, 'M_MRVI': 1, 'Q_MRVI': 1, 'Y_MRVI': 1}
today_stock_df = get_mongo_collection(stock_db, 'today_stock', projection=projection)
history_stock_df = get_mongo_collection(stock_db, 'history_stock', projection=projection, find_query={"date": {"$in": date_series['date'].iloc[:3].tolist()}})
full_stock_df = pd.concat([today_stock_df, history_stock_df])

In [92]:
vol_ratio_df = full_stock_df[['date', 'ticker', 'vol_ratio']].copy()
vol_ratio_df['date'] = vol_ratio_df['date'].astype(str)
vol_ratio_df = vol_ratio_df.pivot(index="ticker", columns="date", values="vol_ratio")
vol_ratio_df['vol_ratio'] = vol_ratio_df.mean(axis=1)
vol_ratio_df = vol_ratio_df.sort_values(by='vol_ratio', ascending=False)
vol_ratio_df.columns.name = None

mtsi_df = full_stock_df[['date', 'ticker', 'W_MTSI', 'M_MTSI', 'Q_MTSI', 'Y_MTSI']].copy()
mtsi_df = mtsi_df[mtsi_df['date'] == today].drop(columns=['date'])
mtsi_df = mtsi_df.set_index('ticker')

mrvi_df = full_stock_df[['date', 'ticker', 'W_MRVI', 'M_MRVI', 'Q_MRVI', 'Y_MRVI']].copy()
mrvi_df = mrvi_df[mrvi_df['date'] == today].drop(columns=['date'])
mrvi_df = mrvi_df.set_index('ticker')

In [97]:
final_df = pd.DataFrame()
for name, stock_list in stock_dict.items():
    temp_df = vol_ratio_df[['vol_ratio']]
    temp_df = temp_df.join(mtsi_df)
    temp_df = temp_df.join(mrvi_df)
    temp_df = temp_df[temp_df.index.isin(stock_list)]

    temp_df = temp_df[temp_df['Y_MTSI'] > 60]
    temp_df = temp_df[temp_df['Q_MTSI'] > 60]
    temp_df = temp_df[(temp_df['M_MTSI'] > 30)]
    temp_df = temp_df[(temp_df['W_MTSI'] > 30)]

    temp_df = temp_df[temp_df['W_MRVI'] > 0.4]
    temp_df = temp_df[temp_df['M_MRVI'] > 0.4]
    temp_df = temp_df[temp_df['Q_MRVI'] > 0.7]
    temp_df = temp_df[temp_df['Y_MRVI'] > 0.7]

    temp_df['group'] = name
    final_df = pd.concat([final_df, temp_df.reset_index()], ignore_index=True)

In [98]:
final_df

,ticker,vol_ratio,W_MTSI,M_MTSI,Q_MTSI,Y_MTSI,W_MRVI,M_MRVI,Q_MRVI,Y_MRVI,group
0,DXS,1.175714,90.000000,76.682674,77.815480,75.582485,0.712270,1.0,0.964776,0.994038,Vượt trội
1,PDR,1.132980,87.751968,80.000000,86.039841,72.419128,0.802062,1.0,1.000000,0.894566,Vượt trội
2,VND,1.074267,81.243406,71.242663,73.330046,72.378605,1.000000,1.0,1.000000,0.715436,Vượt trội
3,CEO,1.049264,66.000000,58.408287,74.319998,73.085713,0.547969,1.0,0.968211,0.948969,Vượt trội
4,MBB,0.950641,66.000000,63.237410,71.942446,71.387461,0.565644,1.0,1.000000,1.000000,Vượt trội
5,MSB,0.906959,98.174366,86.329445,86.916768,82.083406,0.652951,1.0,1.000000,1.000000,Vượt trội
6,VSC,0.879026,76.000000,60.000000,75.750796,74.107711,0.520819,1.0,1.000000,1.000000,Vượt trội
7,VPB,0.872755,67.096480,66.783318,74.069991,72.907136,0.733816,1.0,1.000000,1.000000,Vượt trội
8,SHB,0.872000,68.659635,62.688176,71.612906,71.152075,0.855934,1.0,1.000000,1.000000,Vượt trội
9,SHS,0.867755,80.000000,69.790697,75.874418,74.196013,0.790756,1.0,1.000000,0.707289,Vượt trội
